In [1]:
from keras.layers import Input

# Define input shape to match input data (32, 64, 3)
input_shape = (32, 64, 3)

# Define input layer with input shape
input_layer = Input(shape=input_shape)

# Build the rest of the model
...


Using TensorFlow backend.


Ellipsis

In [2]:
import os
train_dir = "Data/training_set"
uniq_labels = sorted(os.listdir(train_dir))

In [3]:
#can save the images of uniform size 
import cv2
import mediapipe as mp
import time
import numpy as np
from tensorflow.keras.models import load_model

# Initialize mediapipe hands module
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.7, min_tracking_confidence=0.7)

# Initialize mediapipe drawing module
mp_drawing = mp.solutions.drawing_utils 

# Open webcam
cap = cv2.VideoCapture(0)

model = load_model('model_vgg16.h5') 

while True:
    ret, frame = cap.read()
    if not ret:
        print("Unable to capture video")
        break

    # Flip the frame horizontally
    frame = cv2.flip(frame, 1)
    # cv2.imshow("Initial Hand Landmarks", frame)
    # Convert BGR image to RGB before processing
    # frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect hands
    results = hands.process(frame)
    #for initial image 
    image = frame.copy()
    image[:] = (255, 255, 255)
    y_min=0
    x_min=0
    # Draw landmarks on the image
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Draw hand landmarks on the image
            mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Get bounding box of the hand
            x_min = int(min(hand_landmarks.landmark, key=lambda x: x.x).x * image.shape[1])-20
            y_min = int(min(hand_landmarks.landmark, key=lambda x: x.y).y * image.shape[0])-20
            x_max = int(max(hand_landmarks.landmark, key=lambda x: x.x).x * image.shape[1])+20
            y_max = int(max(hand_landmarks.landmark, key=lambda x: x.y).y * image.shape[0])+20

            # Draw rectangle around the hand
            cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
    
    p_img = 'X'
    if y_min>0 and x_min>0:#condition for error handling
        e_img = image[y_min:y_max,x_min:x_max] #store the required image
        e_img = cv2.resize(e_img, (64, 64))
        e_img = e_img / 255.0
        e_img = e_img.reshape((1, 64, 64, 3))
        p_img = model.predict(e_img)
        p_img = np.argmax(p_img, axis=1)
        
        # Add text box to the image
        text = uniq_labels[p_img[0]] #predicted alphabet to be kept
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 1
        thickness = 2
        color = (0, 255, 0)
        x, y = x_min, y_min # coordinates of the top-left corner of the text box
        cv2.putText(frame, text, (x, y), font, font_scale, color, thickness)
    cv2.imshow("Original Hand Landmarks", frame)
    # Wait for key press and exit if 'q' is pressed
    key = cv2.waitKey(1)
    if key == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()
